# AI Study Companion - Multi-Agent Learning System

## Overview
This notebook demonstrates a multi-agent AI system for personalized education using:
- **3 Agents**: Planner → Researcher → Tutor
- **2 Tools**: Google Search + Code Execution
- **Memory**: Session-based learning progress tracking

**Track**: Agents for Good (Education-focused)

## 🔧 Setup & Installation

In [ ]:
# Install required packages
!pip install -q google-genai python-dotenv

In [ ]:
import os
from google import genai
from google.genai import types
import json
from datetime import datetime

# Set your API key here
os.environ['GOOGLE_API_KEY'] = 'YOUR_API_KEY_HERE'  # Replace with your actual API key
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))

## Agent Definitions

In [ ]:
def create_planner_agent(client):
    """
    Planner Agent: Breaks down learning goals into structured plans
    """
    system_instruction = """
    You are an expert Learning Planner Agent specializing in educational curriculum design.
    
    Your role:
    1. Analyze the user's learning query and current knowledge level
    2. Break down complex topics into digestible learning modules
    3. Create a structured learning path with clear objectives
    4. Consider prerequisite knowledge and suggest starting points
    5. Estimate time required for each module
    
    Output format:
    - Learning Objectives (3-5 clear goals)
    - Prerequisites needed
    - Module breakdown (step-by-step)
    - Recommended learning order
    - Estimated time per module
    
    Be concise, structured, and pedagogically sound.
    """
    
    return {
        'name': 'planner_agent',
        'system_instruction': system_instruction,
        'role': 'learning_planner'
    }

print("✅ Planner Agent defined")

In [ ]:
def create_researcher_agent(client):
    """
    Researcher Agent: Gathers information using Google Search tool
    """
    system_instruction = """
    You are an expert Research Agent specializing in gathering educational content.
    
    Your role:
    1. Use Google Search to find accurate, up-to-date information
    2. Identify authoritative sources (educational sites, documentation, papers)
    3. Extract key concepts, definitions, and examples
    4. Verify information across multiple sources
    5. Summarize findings in a structured format
    
    Focus on:
    - Technical accuracy
    - Recent information (when applicable)
    - Diverse perspectives and examples
    - Practical applications
    
    Always cite your sources and prioritize educational quality.
    """
    
    # Google Search tool
    google_search_tool = types.Tool(
        google_search=types.GoogleSearch()
    )
    
    return {
        'name': 'researcher_agent',
        'system_instruction': system_instruction,
        'tools': google_search_tool,
        'role': 'information_researcher'
    }

print("✅ Researcher Agent defined")

In [ ]:
def create_tutor_agent(client):
    """
    Tutor Agent: Creates personalized lessons with code examples
    """
    system_instruction = """
    You are an expert AI Tutor Agent specializing in personalized education.
    
    Your role:
    1. Create comprehensive, engaging lessons based on research data
    2. Adapt content to student's knowledge level
    3. Include practical examples and code demonstrations
    4. Use the code execution tool to validate examples
    5. Provide exercises and knowledge checks
    6. Explain complex concepts with analogies
    
    Lesson structure:
    1. Introduction & Context (why this matters)
    2. Core Concepts (detailed explanations)
    3. Practical Examples (with working code)
    4. Common Pitfalls & Tips
    5. Practice Exercises
    6. Further Resources
    
    Teaching style:
    - Clear and encouraging
    - Build from fundamentals
    - Use real-world examples
    - Interactive and engaging
    """
    
    # Code execution tool
    code_execution_tool = types.Tool(
        code_execution=types.CodeExecution()
    )
    
    return {
        'name': 'tutor_agent',
        'system_instruction': system_instruction,
        'tools': code_execution_tool,
        'role': 'personalized_tutor'
    }

print("✅ Tutor Agent defined")

## Session Manager (Memory)

In [ ]:
class SessionManager:
    """
    Manages user learning sessions and memory persistence.
    Implements short-term memory for the agent system.
    """
    
    def __init__(self, storage_path='./sessions'):
        self.storage_path = storage_path
        os.makedirs(storage_path, exist_ok=True)
    
    def get_session(self, session_id: str) -> dict:
        """Load existing session or create new one"""
        session_file = os.path.join(self.storage_path, f'{session_id}.json')
        
        if os.path.exists(session_file):
            with open(session_file, 'r') as f:
                return json.load(f)
        
        # New session
        return {
            'session_id': session_id,
            'created_at': datetime.now().isoformat(),
            'topics_covered': [],
            'level': 'beginner',
            'interactions': []
        }
    
    def save_session(self, session_id: str, session_data: dict):
        """Persist session data"""
        session_data['updated_at'] = datetime.now().isoformat()
        session_file = os.path.join(self.storage_path, f'{session_id}.json')
        
        with open(session_file, 'w') as f:
            json.dump(session_data, f, indent=2)
    
    def get_learning_history(self, session_id: str) -> list:
        """Get all topics covered by user"""
        session = self.get_session(session_id)
        return session.get('topics_covered', [])

# Initialize session manager
session_manager = SessionManager()
print("✅ Session Manager initialized")

## Main Multi-Agent Orchestration

In [ ]:
def run_study_companion(user_query: str, session_id: str = "default"):
    """
    Main multi-agent orchestration function.
    Flow: User Query → Planner → Researcher → Tutor → Response
    """
    
    # Load session memory
    session_data = session_manager.get_session(session_id)
    
    print(f"\n{'='*60}")
    print(f"📚 AI Study Companion - Processing Query")
    print(f"{'='*60}\n")
    
    # Step 1: Planner Agent - Breaks down learning goal
    print("🧠 Step 1: Planning learning path...")
    planner_agent = create_planner_agent(client)
    
    planner_context = f"""
    User Query: {user_query}
    Previous Topics Covered: {session_data.get('topics_covered', [])}
    Current Level: {session_data.get('level', 'beginner')}
    """
    
    planner_response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=planner_context,
        config=types.GenerateContentConfig(
            system_instruction=planner_agent['system_instruction'],
            temperature=0.7
        )
    )
    
    learning_plan = planner_response.text
    print(f"✅ Learning Plan Created:\n{learning_plan[:200]}...\n")
    
    # Step 2: Researcher Agent - Gathers information using search
    print("🔍 Step 2: Researching topic...")
    researcher_agent = create_researcher_agent(client)
    
    research_response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=f"Research this topic thoroughly: {user_query}",
        config=types.GenerateContentConfig(
            system_instruction=researcher_agent['system_instruction'],
            tools=[researcher_agent['tools']],
            temperature=0.5
        )
    )
    
    research_data = research_response.text
    print(f"✅ Research Completed\n")
    
    # Step 3: Tutor Agent - Creates personalized lesson
    print("👨‍🏫 Step 3: Creating personalized lesson...")
    tutor_agent = create_tutor_agent(client)
    
    tutor_context = f"""
    Learning Plan: {learning_plan}
    Research Data: {research_data}
    Student Level: {session_data.get('level', 'beginner')}
    Previous Knowledge: {session_data.get('topics_covered', [])}
    
    Create a comprehensive lesson for: {user_query}
    """
    
    tutor_response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=tutor_context,
        config=types.GenerateContentConfig(
            system_instruction=tutor_agent['system_instruction'],
            tools=[tutor_agent['tools']],  # Code execution for examples
            temperature=0.8
        )
    )
    
    final_lesson = tutor_response.text
    print(f"✅ Lesson Created\n")
    
    # Update session memory
    session_data['topics_covered'] = session_data.get('topics_covered', []) + [user_query]
    session_data['last_query'] = user_query
    session_manager.save_session(session_id, session_data)
    
    print(f"\n{'='*60}")
    print(f"📖 FINAL LESSON")
    print(f"{'='*60}\n")
    print(final_lesson)
    
    return {
        'plan': learning_plan,
        'research': research_data,
        'lesson': final_lesson,
        'session_id': session_id
    }

print("✅ Main orchestration function defined")

## Run the AI Study Companion

In [ ]:
# Example 1: Binary Search Trees
result = run_study_companion(
    user_query="Explain binary search trees with Python code examples",
    session_id="demo_user"
)

In [ ]:
# Example 2: Neural Networks (builds on previous session)
result2 = run_study_companion(
    user_query="Teach me the basics of neural networks",
    session_id="demo_user"
)

## View Session Memory

In [ ]:
# Check learning history
session_data = session_manager.get_session("demo_user")
print("\n📚 Learning History:")
print(json.dumps(session_data, indent=2))

## Summary

This notebook demonstrates:

### Multi-Agent System
- **Planner Agent**: Creates structured learning paths
- **Researcher Agent**: Gathers information via Google Search
- **Tutor Agent**: Generates personalized lessons with code

### Tools Integration
- **Google Search**: Real-time information retrieval
- **Code Execution**: Live code examples and validation

### Memory Management
- **Session Persistence**: Tracks learning progress
- **Context Awareness**: Builds on previous topics
- **User Profiles**: Maintains skill level and history

### Track: Agents for Good
This project addresses educational accessibility by providing personalized, AI-powered tutoring to anyone with internet access.

---

